In [101]:
import os
import sys
import random
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk import word_tokenize
import langdetect
from langdetect import detect
import re
from nltk.metrics import precision, recall, f_measure
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy
from sklearn.feature_extraction.text import TfidfVectorizer
def document_features(tokens, word_features, tfidf_vectorizer, n = 2):
    doc_words = set(tokens)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in doc_words)
    return features
    #tfidf score calculation
    tfidf_scores = tfidf_vectorizer.transform([' '.join(tokens)])
    for feature, score in zip(word_features, tfidf_scores.data):
        features[feature] = score
    return features
    #ngrams for word patterns
    ngrams = [tuple(tokens[i:i + n]) for i in range(len(tokens) - n + 1)]
    for gram in ngrams:
        features['ngram_{}'.format('_'.join(gram))] = True
    return features
    #find URLs
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z0-9]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    has_url = any(url_pattern.search(word) for word in tokens)
    features['contains_url'] = has_url
    return features
    #negation representation
    for x in range(len(tokens)):
        if x > 0 and document[x-1].lower() in ['not', 'no', 'never', 'neither', 'nobody', 'none', 'nor', 'nothing', 'nowhere']:
            features['negation_{}'.format(tokens[x].lower())] = True
    return features
    #POS tags
    pos_tags = nltk.pos_tag(tokens)
    for tag in pos_tags:
        features['pos_tag_{}'.format(tag[1])] = True
    return features
#finding the top 20 used words
def get_top_words(emaildocs, num_words=20):
    all_words = FreqDist(word for tokens, _ in emaildocs for word in tokens)
    top_words = [word for word, _ in all_words.most_common(num_words)]
    return top_words
#making sure tokens are English
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False
#filtering tokens that aren't real words
def filter_gibberish(tokens, min_word_length=3):
    return [word for word in tokens if len(word) >= min_word_length and is_english(word)]
#split tokens that seem to be combined words
def split_combined_words(tokens):
    return [subtoken.lower() for token in tokens for subtoken in re.findall(r'\b\w+\b', token)]
# function to read spam and ham files, train and test a classifier 
def processspamham(dirPath, limitStr):
  # convert the limit argument from a string to an int
  limit = int(limitStr)
  
  # start lists for spam and ham email texts
  hamtexts = []
  spamtexts = []
  os.chdir(dirPath)
  # process all files in directory that end in .txt up to the limit
  #    assuming that the emails are sufficiently randomized
  for file in os.listdir("./spam"):
    if (file.endswith(".txt")) and (len(spamtexts) < limit):
      # open file for reading and read entire file into a string
      f = open("./spam/"+file, 'r', encoding="latin-1")
      spamtexts.append (f.read())
      f.close()
  for file in os.listdir("./ham"):
    if (file.endswith(".txt")) and (len(hamtexts) < limit):
      # open file for reading and read entire file into a string
      f = open("./ham/"+file, 'r', encoding="latin-1")
      hamtexts.append (f.read())
      f.close()
  
  # print number emails read
  print ("Number of spam files:",len(spamtexts))
  print ("Number of ham files:",len(hamtexts))
  print
  
  # create list of mixed spam and ham email documents as (list of words, label)
  emaildocs = []
  # add all the spam
  for spam in spamtexts:
    tokens = nltk.word_tokenize(spam)
    emaildocs.append((tokens, 'spam'))
  # add all the regular emails
  for ham in hamtexts:
    tokens = nltk.word_tokenize(ham)
    emaildocs.append((tokens, 'ham'))
  
  # randomize the list
  random.shuffle(emaildocs)

  # possibly filter tokens
  stop_words = set(stopwords.words('english'))
  emaildocs = [([word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words], label) for tokens, label in emaildocs]
  
  # continue as usual to get all words and create word features
  all_words = FreqDist(word for tokens, _ in emaildocs for word in tokens)

  #filtering out gibberish and combined words
  for spam in spamtexts:
        if is_english(spam):  # Filter out non-English text
            tokens = nltk.word_tokenize(spam)
            #filter out potential gibberish and combined words
            filtered_tokens = split_combined_words(filter_gibberish(tokens))
            emaildocs.append((filtered_tokens, 'spam'))
    
  for ham in hamtexts:
        if is_english(ham):  # Filter out non-English text
            tokens = nltk.word_tokenize(ham)
            # Filter out short words and non-English words (potential gibberish)
            filtered_tokens = split_combined_words(filter_gibberish(tokens))
            emaildocs.append((filtered_tokens, 'ham'))

  #find the top 20 most used words
  top_words = [word for word, _ in all_words.most_common(20)]

  # feature sets from a feature definition function
  word_features = list(all_words.keys())[:3000]
  all_texts = [' '.join(tokens) for tokens, _ in emaildocs]
  tfidf_vectorizer = TfidfVectorizer(stop_words='english')
  featuresets = [(document_features(tokens, word_features, tfidf_vectorizer), label) for tokens, label in emaildocs]

  #tfidf feature-sets and vectorization
  
 # tfidf_vectorizer.fit(all_texts)
  emaildocs_tfidf = [([word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words], label) for tokens, label in emaildocs]
  all_words_tfidf = FreqDist(word for tokens, _ in emaildocs_tfidf for word in tokens)
  top_words_tfidf = [word for word, _ in all_words_tfidf.most_common(50)]
  featuresets_tfidf = [(document_features(tokens, top_words_tfidf, tfidf_vectorizer), label) for tokens, label in emaildocs_tfidf]

  # tfidf cross-validation
  num_folds_tfidf = 5
  fold_size_tfidf = len(featuresets_tfidf) // num_folds_tfidf

  avg_precision_tfidf = 0
  avg_recall_tfidf = 0
  avg_f_measure_tfidf = 0

  for i in range(num_folds_tfidf):
        start_idx = i * fold_size_tfidf
        end_idx = (i + 1) * fold_size_tfidf

        testing_set_tfidf = featuresets_tfidf[start_idx:end_idx]
        training_set_tfidf = featuresets_tfidf[:start_idx] + featuresets_tfidf[end_idx:]

        classifier_tfidf = NaiveBayesClassifier.train(training_set_tfidf)

        # Evaluate the classifier on the testing set
        reference_set_tfidf = set(label for _, label in testing_set_tfidf)
        test_set_tfidf = set(classifier_tfidf.classify(features) for features, _ in testing_set_tfidf)

        precision_tfidf = precision(reference_set_tfidf, test_set_tfidf)
        recall_tfidf = recall(reference_set_tfidf, test_set_tfidf)
        f_measure_tfidf = f_measure(reference_set_tfidf, test_set_tfidf)

        avg_precision_tfidf += precision_tfidf
        avg_recall_tfidf += recall_tfidf
        avg_f_measure_tfidf += f_measure_tfidf
      
  # calculate the average scores for tfidf
  avg_precision_tfidf /= num_folds_tfidf
  avg_recall_tfidf /= num_folds_tfidf
  avg_f_measure_tfidf /= num_folds_tfidf

  #print tfidf results
  print("tfidf Average Precision:", avg_precision_tfidf)
  print("tfidf Average Recall:", avg_recall_tfidf)
  print("tfidf Average F-measure:", avg_f_measure_tfidf)
  
  # train classifier and show performance in cross-validation
  num_folds = 5
  subset_size = int(len(featuresets) / num_folds)
  precision_scores = []
  recall_scores = []
  f_measure_scores = []

  for i in range(num_folds):
      testing_set = featuresets[i * subset_size:][:subset_size]
      training_set = featuresets[:i * subset_size] + featuresets[(i + 1) * subset_size:]

      classifier = NaiveBayesClassifier.train(training_set)

    # Evaluate the classifier on the testing set
      reference_set = set(label for _, label in testing_set)
      test_set = [(document_features(features, word_features, tfidf_vectorizer), label) for features, label in testing_set]
    
      precision_scores.append(precision(reference_set, set(classifier.classify(features) for features, _ in test_set)))
      recall_scores.append(recall(reference_set, set(classifier.classify(features) for features, _ in test_set)))
      f_measure_scores.append(f_measure(reference_set, set(classifier.classify(features) for features, _ in test_set)))

    # Print N-gram features for each iteration
      for features, label in test_set[:4]:
          print("Label: {}".format(label))

    # Check for ngrams features
          ngram_features = {feature: value for feature, value in features.items() if 'ngram_' in feature}

          if ngram_features:
              print("N-gram Features:")
              print(ngram_features)
          #else:
              #print("No N-gram Features found.")

    #print URL, if found
      for features, label in testing_set:
          if features.get('contains_url'):
              print(f"URLs found in '{label}':")
              for url in features.get('urls_found', []):
                  print(f" - {url}")
          #else:
              #print("No URL found.")

      #Negation representation
      for examples, label in testing_set[:4]:
          print("Negation Representation for '{}': ".format(label))
          for feature, value in examples.items():
              if feature.startswith('negation_') and value:
                  print(feature)
        #POS tags
      for examples, label in testing_set[:4]:
          print("POS Tag Features for '{}': ".format(label))
          for feature, value in examples.items():
              if feature.startswith('pos_tag_') and value:
                  print(feature)

      #Find common spam words using tfidf
      #read the emails
      #spam_texts = []
      #for file in os.listdir(os.path.join(dirPath, "spam")):
      #    if file.endswith(".txt"):
       #       with open(os.path.join(dirPath, "spam", file), 'r', encoding="latin-1") as f:
      #            spam_texts.append(f.read())
      #find spam keywords using tfidf
      #tfidf_matrix = tfidf_vectorizer.fit_transform(spam_texts)
      #get words with their tfidf scores
      #spam_words = tfidf_vectorizer.get_feature_names_out()
      #spam_tfidf_scores = tfidf_matrix.sum(axis=0).A1
      #make a dictionary containing the spam words
      #keyword_dict = dict(zip(spam_texts, spam_tfidf_scores))
      #print the top words
      #num_spam = 10
      #top_spam = sorted(keyword_dict.items(), key=lambda x: x[1], reverse=True)[:num_spam]

      #print("Top Keywords Associated with Spam:")
      #for word, _ in top_spam:
       #   print(word)

                  
  #calculate the average scores
  avg_precision = sum(precision_scores) / num_folds  
  avg_recall = sum(recall_scores) / num_folds
  avg_f_measure = sum(f_measure_scores) / num_folds

  #prints the average scores
  print('Avg Precision = ', avg_precision)
  print('Avg Recall = ', avg_recall)
  print('Avg F-Measure = ', avg_f_measure)
  

  #prints the top 50 words
  print('Top Words: ')
  for word in top_words:
      print(f" - {word}")

  #finds and prints given spam words
  spam_filter = ['free','guarantee','opportunity','viagra','xanax','sex','miracle','click','call now','act now','earn','winner','pe','nis','ize','mat','ters','yhvqbvdboevkcd','money','purchase','satisfied','results','product','products','levitra','government','officials','partner']
  spam_filter = [word.lower() for word in spam_filter]
  print("Spam words found:")
  for tokens, label in emaildocs:
        if any(word.lower() in [t.lower() for t in tokens] for word in spam_filter):
            print(f"{label}: {' '.join(tokens)}")
      
  # print a few token lists
  for email in emaildocs[:4]:
    print (email)
    print

In [102]:
processspamham(r'C:\Users\antho\OneDrive\Documents\Syracuse\664 - Natural Language Processing\Homework\Final Project\EmailSpamCorpora\corpus', '20')

Number of spam files: 20
Number of ham files: 20
tfidf Average Precision: 0.9
tfidf Average Recall: 0.9
tfidf Average F-measure: 0.8666666666666666
Label: spam
Label: ham
Label: spam
Label: ham
Negation Representation for 'spam': 
Negation Representation for 'ham': 
Negation Representation for 'spam': 
Negation Representation for 'ham': 
POS Tag Features for 'spam': 
POS Tag Features for 'ham': 
POS Tag Features for 'spam': 
POS Tag Features for 'ham': 
Label: spam
Label: ham
Label: spam
Label: spam
Negation Representation for 'spam': 
Negation Representation for 'ham': 
Negation Representation for 'spam': 
Negation Representation for 'spam': 
POS Tag Features for 'spam': 
POS Tag Features for 'ham': 
POS Tag Features for 'spam': 
POS Tag Features for 'spam': 
Label: spam
Label: spam
Label: spam
Label: spam
Negation Representation for 'spam': 
Negation Representation for 'spam': 
Negation Representation for 'spam': 
Negation Representation for 'spam': 
POS Tag Features for 'spam': 
POS